<a href="https://colab.research.google.com/github/jamesdievas/smartIrrigation/blob/main/SimulaIrrigacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
const int trigPin = 9;
const int echoPin = 10;
const int irrigacaoPin = 7;

long duracao;
int distancia;

// Duração da irrigação em ms
const unsigned long duracaoLigado = 60000UL; // 1 minuto

// Horários relativos ao início da simulação
const unsigned long horario1 = 60000UL;  // 1 min após iniciar
const unsigned long horario2 = 120000UL; // 2 min após iniciar (simula 06:00)
const unsigned long horario3 = 240000UL; // 4 min após iniciar (simula 18:00)

unsigned long tempoInicial;
unsigned long fimIrrigacaoAtual = 0; // Final do período de irrigação
bool irrigacaoAnterior = false;      // Para detectar mudança de status
bool pausadaSensor = false;          // Indica se a irrigação está pausada pelo sensor

void setup() {
  pinMode(trigPin, OUTPUT);
  pinMode(echoPin, INPUT);
  pinMode(irrigacaoPin, OUTPUT);
  Serial.begin(9600);
  digitalWrite(irrigacaoPin, LOW);

  tempoInicial = millis();
}

void loop() {
  unsigned long agora = millis();
  unsigned long tempoDesdeInicio = agora - tempoInicial;

  // ---- Sensor ultrassônico ----
  digitalWrite(trigPin, LOW);
  delayMicroseconds(2);
  digitalWrite(trigPin, HIGH);
  delayMicroseconds(10);
  digitalWrite(trigPin, LOW);

  duracao = pulseIn(echoPin, HIGH, 20000);
  if (duracao == 0) {
    distancia = 999;
  } else {
    distancia = duracao * 0.034 / 2;
  }

  // ---- Controle de irrigação ----
  bool dentroHorario = false;

  // Verifica cada horário
  if (tempoDesdeInicio >= horario1 && tempoDesdeInicio < horario1 + duracaoLigado) {
    dentroHorario = true;
    fimIrrigacaoAtual = horario1 + duracaoLigado;
  } else if (tempoDesdeInicio >= horario2 && tempoDesdeInicio < horario2 + duracaoLigado) {
    dentroHorario = true;
    fimIrrigacaoAtual = horario2 + duracaoLigado;
  } else if (tempoDesdeInicio >= horario3 && tempoDesdeInicio < horario3 + duracaoLigado) {
    dentroHorario = true;
    fimIrrigacaoAtual = horario3 + duracaoLigado;
  }

  // Decide se a irrigação deve estar ligada
  bool irrigar = dentroHorario && (distancia > 200);

  // Indica se está pausada pelo sensor
  pausadaSensor = dentroHorario && (distancia <= 200);

  digitalWrite(irrigacaoPin, irrigar ? HIGH : LOW);

  // ---- Mostrar mensagem apenas se o status mudou ----
  if (irrigar != irrigacaoAnterior) {
    irrigacaoAnterior = irrigar;

    // Converter tempoDesdeInicio para hh:mm:ss
    unsigned long totalSegundos = tempoDesdeInicio / 1000;
    int horas = totalSegundos / 3600;
    int minutos = (totalSegundos % 3600) / 60;
    int segundos = totalSegundos % 60;

    Serial.println("------------------------");
    Serial.print("Horario atual: ");
    if (horas < 10) Serial.print("0");
    Serial.print(horas);
    Serial.print(":");
    if (minutos < 10) Serial.print("0");
    Serial.print(minutos);
    Serial.print(":");
    if (segundos < 10) Serial.print("0");
    Serial.print(segundos);

    if (irrigar) {
      Serial.println(" - Status: LIGADO (Irrigacao INICIADA)");
    } else {
      if (pausadaSensor) {
        Serial.println(" - Status: DESLIGADO (Irrigacao PAUSADA pelo sensor)");
      } else {
        Serial.println(" - Status: DESLIGADO (Irrigacao FINALIZADA)");
      }
    }
  }

  // ---- Contador regressivo da irrigacao (mm:ss) ----
  if (dentroHorario) {
    unsigned long restanteMs = fimIrrigacaoAtual - tempoDesdeInicio;
    if (restanteMs > duracaoLigado) restanteMs = duracaoLigado; // Protege limites
    unsigned long restanteSeg = restanteMs / 1000;
    int restanteMin = restanteSeg / 60;
    int restanteSec = restanteSeg % 60;

    Serial.print("Tempo restante da irrigacao: ");
    if (restanteMin < 10) Serial.print("0");
    Serial.print(restanteMin);
    Serial.print(":");
    if (restanteSec < 10) Serial.print("0");
    Serial.println(restanteSec);
  }

  // ---- Mostrar distancia ----
  Serial.print("Distancia: ");
  Serial.print(distancia);
  Serial.println(" cm");

  delay(500);
}

SyntaxError: invalid decimal literal (ipython-input-2214199870.py, line 9)